In [6]:
import sys

sys.path.append('../')
sys.path.append('../my_libs/')
sys.path.append('../my_libs/img/')

!{sys.executable} -m pip install -r ../../pip-dependencies.txt

Defaulting to user installation because normal site-packages is not writeable


In [7]:
from my_libs.tools import *
from my_libs.colors import *

## Cvičení

Natrénujte a vyhodnoťte klasifikátor založený na hluboké neuronové síti, který 
bude  z  obrázku  člověka  určovat,  zda  se  jedná  o  dospělého  či  o  dítě.  Pro 
trénování a validaci využijte dataset children-vs-adults.zip. Klasifikátor by měl 
dosáhnout přesnosti (accuracy) alespoň 80 % na validační množině. 

### Poznámky
Není to nezbytné, ale běh na GPU bude rychlejší než na CPU. Kód lze spustit na zdarma dostupných službách poskytujících GPU, např.: 
- [Google colab](https://colab.research.google.com/drive/16pBJQePbqkz3QFV54L4NIkOn1kwpuRrj) 

Můžete  použít  libovolnou  dostupnou  knihovnu  pro  práci  s  neuronovými sítěmi.  Doporučené  jsou PyTorch nebo Keras, které  jsou  dobře dokumentované se spoustou příkladů. 

Doporučený postup je tzv. transfer learning, tj. použití existujícího modelu a  jeho  přizpůsobení  na  novou  úlohu.  Příklady  transfer  learningu  jsou dostupné na Transfer learning v:
- [Pytorch](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)
- [Keras](https://keras.io/guides/transfer_learning/)

Předtrénované  neuronové  sítě  pro  klasifikaci  obrázků  lze  vybírat  z  tzv. 
ZOO modelů. 
Knihovna modelů a datasetů:
- [torchvision](https://pytorch.org/vision/stable/index.html)
- [Pytorch timm](https://huggingface.co/docs/timm/quickstart)
- [Keras](https://keras.io/api/applications/)

Není nezbytné používat veké modely. Požadovaného skóre lze dosáhnout i s menšími  modely. Např.  předtrénovaný model  „resnet10t”  z  knihovny timm (PyTorch) dosahuje 80% přesnosti  již  po  `3`  epochách při `batch_size=100`  a  optimalizačním algoritmu  Adam  s  `learning_rate = 0.001`. 
Jedna epocha na CPU Ryzen 7 5700X trvá cca 21 s.

## Uvažované modely
Stažené z [Hugging Face](https://huggingface.co/models?pipeline_tag=image-classification&sort=trending&search=Microsoft+resnet)

Z dílny Microsoftu:
- [resnet-18](https://huggingface.co/microsoft/resnet-18)
- [resnet-50](https://huggingface.co/microsoft/resnet-50)
- [resnet-152](https://huggingface.co/microsoft/resnet-152)

### Výběr modelu a jeho modifikace

In [ ]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
import os
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [9]:
# Načtení modelu a procesoru
image_processor = AutoImageProcessor.from_pretrained("microsoft/resnet-18")
model = AutoModelForImageClassification.from_pretrained("microsoft/resnet-18")

# Nastavení modelu pro transfer learning
if isinstance(model.classifier, torch.nn.Sequential):
    in_features = model.classifier[-1].in_features
    model.classifier[-1] = torch.nn.Linear(in_features, 2)  # Binary classification (adult/children)
else:
    in_features = model.classifier.in_features
    model.classifier = torch.nn.Linear(in_features, 2)  # Binary classification (adult/children)


### Hyper parametry

In [ ]:
# parametry
batch_size = 100
learning_rate = 1e-4
num_epochs = 10

## Data set

In [ ]:
number_of_pictures = 800
folder = "children_vs_adults"
train_folder = "train"
test_folder = "valid"

# Cesty k trénovacím a testovacím složkám
train_path = os.path.join(folder, train_folder)
test_path = os.path.join(folder, test_folder)

In [ ]:
# Nastavení transformací pro dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
])

# Načtení datasetu
print("Loading the dataset...")
print("Train path: ", Blue + train_path + NC)
print("Test path: ", Blue + test_path + NC)

# Načtení datasetu
train_dataset = datasets.ImageFolder(train_path, transform=transform)
test_dataset = datasets.ImageFolder(test_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("Number of training examples: ", Blue + str(len(train_dataset)) + NC)
print("Number of test examples: ", Blue + str(len(test_dataset)) + NC)

print("")
if len(train_dataset) + len(test_dataset) != number_of_pictures:
    print("Number of pictures in the dataset is not correct.")
    print("Expected number of pictures: ", number_of_pictures)
    print("Number of pictures in the dataset: ", len(train_dataset) + len(test_dataset))
    exit(1)
else:
    print(Green + "Dataset loaded successfully." + NC)


Loading the dataset...
Train path:  children_vs_adults/train
Test path:  children_vs_adults/valid
Number of training examples:  680
Number of test examples:  120

Dataset loaded successfully.


## Transfer learning

### Nastavení trénování

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


print("Hyper parameters:")
print("Batch size: \t", Blue + str(batch_size) + NC)
print("Learning rate: \t", Blue + str(learning_rate) + NC)
print("Usig device: \t", Blue + str(device) + NC)

Hyper parameters:
Batch size: 	 8
Learning rate: 	 0.0001
Usig device: 	 cpu


### Trénování modelu

In [ ]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Vyhodnocení modelu
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images).logits
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of the model on the test images: {100 * correct / total:.2f}%")

KeyboardInterrupt: 

In [ ]:
# plot loss
plt.figure()
plt.plot(running_loss)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss")
plt.show()